In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [2]:
train_data = np.load('train_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

test_data = np.load('test_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

train = np.zeros([2000, 150, 150, 3])
for i in range(2000):
    train[i] = train_data[i]
    
test = np.zeros([800, 150, 150, 3])
for i in range(800):
    test[i] = test_data[i]

In [3]:
batch_size = 16

In [12]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)
    
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = datagen.flow(train, 
                             batch_size=batch_size,
                             shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, 2000 // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    
    generator = datagen.flow(test, 
                             batch_size=batch_size, 
                             shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(generator, 800 // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [25]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array([0] * 1000 + [1] * 1000)
    
    test_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    test_labels = np.array([0] * 400 + [1] * 400)
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    
    model.fit(train_data,
              train_labels, 
              epochs=50,
              batch_size=batch_size,
              validation_data=(test_data, test_labels))
    model.save_weights('bottleneck_fc_model.h5')

In [17]:
save_bottleneck_features()

KeyboardInterrupt: 

In [26]:
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.8874 - acc: 0.7615 - val_loss: 0.2961 - val_acc: 0.8788
Epoch 2/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.3595 - acc: 0.8555 - val_loss: 0.2691 - val_acc: 0.8938
Epoch 3/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.2770 - acc: 0.8895 - val_loss: 0.3111 - val_acc: 0.8662
Epoch 4/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.2359 - acc: 0.9085 - val_loss: 0.3062 - val_acc: 0.8838
Epoch 5/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.1910 - acc: 0.9280 - val_loss: 0.3069 - val_acc: 0.8912
Epoch 6/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.1872 - acc: 0.9280 - val_loss: 0.4800 - val_acc: 0.8438
Epoch 7/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.1704 - acc: 0.9350 - val_loss: 0.2850 - val_acc: 0.9012
Epoch 8/50
2000